In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

from mushroom_rl.core import Environment, MDPInfo
from mushroom_rl.utils import spaces
from mushroom_rl.utils.viewer import Viewer

/home/fantasticoven/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/fantasticoven/.local/lib/python3.8/site-packages/gym/envs/registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(
/home/fantasticoven/.local/lib/python3.8/site-packages/gym/envs/registration.py:407: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
pybullet build time: May 20 2022 19:44:17


In [7]:
class SimEnv(Environment):
  
    def __init__(self):
        self.time_step = 1
        self.offset = 10

        self._viewer = Viewer(env_width = 22, env_height = 22, background = (0, 0, 0))
        
        self.action_space = spaces.Box(np.array([-1, -1]), np.array([1, 1]))
        self.observation_space = spaces.Box(np.array([-10, -10, 0]), np.array([10, 10, 1]))
    
        self.state = np.array([0.0, 0.0])
        self.intersection = np.array([10.0, 10.0])
        self.goal = np.array([10.0, 10.0])

        self.next_manifold = False

    def _reward(self, state):
        state_1, state_2 = state[0], state[1]

        ### Fall off the manifold ###
        if state_2 != 0 and state_1 != 0:
            # Negative of l2 distance to current manifold
            if self.next_manifold:
                reward = -np.abs(state_1 - 10)
            else:
                reward = -np.abs(state_2)
            done = True
        
        ### Reach the interseciton ###
        elif np.array_equal(state, self.goal):
            if self.next_manifold:
                reward = -np.linalg.norm(state - self.goal)
            else:
                reward = 10

        ### Reach the goal ###
        elif np.array_equal(state, self.goal):
            reward = 100
            done = True

        ### Direct the agent to intersection point / goal
        else: 
            if self.next_manifold:
                reward = -np.linalg.norm(self.state - self.goal)
            else: 
                reward = -np.linalg.norm(self.state - self.intersection)
        return reward

    def step(self, action):
        ### Projection ###
        if self.next_manifold: 
            _action = np.array([0.0, action[1]])
        else:
            _action = np.array([action[0], 0.0])

        state_1 += self.state[0] + self.action[0]
        state_2 += self.state[1] + self.action[1]
        state = np.array([state_1, state_2]) 

        done = False
        
        reward = self._reward((state_1, state_2))

        self.state[0] += _action[0]
        self.state[1] += _action[1]

        if self.state[0] == 10:
            self.next_manifold = True

        info = {}
        return self.state, reward, done, info

    def render(self):
        start = np.array([10.0, 10.0])
        intersection = np.array([20.0, 10.0])
        goal = np.array([20.0, 20.0])
        agent = np.array([10.0, 10.0])

        hline_start = np.array([0.0, 10])
        hline_end = np.array([22, 10])
        vline_start = np.array([20, 0])
        vline_end = np.array([20, 22])
        self._viewer.line(hline_start, hline_end, color = (100, 100, 100), width = 5)
        self._viewer.line(vline_start, vline_end, color = (100, 100, 100), width = 5)
        self._viewer.circle(center = start, radius = 0.3, color = (255, 0, 0))
        self._viewer.circle(center = intersection, radius = 0.3, color = (0, 0, 255))
        self._viewer.circle(center = goal, radius = 0.3, color = (0, 255, 0))
        self._viewer.circle(center = agent + self.state, radius = 0.3, color = (50, 100, 150))
        self._viewer.display(self.time_step)
        self._viewer.close()

    def resest(self):
        self.state = np.array([0.0, 0.0])
        self.next_manifold = False
        return self.state


In [10]:
env = SimEnv()
for _ in range(10):
    env.state[0] += 1
    env.render()